In [ ]:
import pandas as pd
amazon = pd.read_csv("/Users/jungsujin/PADA_LAB/구조방정식/Count_SEM/scaled_data/amazon_scaled.csv")
audible = pd.read_csv("/Users/jungsujin/PADA_LAB/구조방정식/Count_SEM/scaled_data/audible_scaled.csv")
coursera = pd.read_csv("/Users/jungsujin/PADA_LAB/구조방정식/Count_SEM/scaled_data/coursera_scaled.csv")
hotel = pd.read_csv("/Users/jungsujin/PADA_ LAB/구조방정식/Count_SEM/scaled_data/hotel_scaled.csv")

In [14]:
import numpy as np
from semopy import Model
import statsmodels.api as sm
import statsmodels.formula.api as smf

ModuleNotFoundError: No module named 'semopy'

In [7]:
# -------------------------------------------------
# (1단계) 측정모델(CFA)을 semopy로 적합
# -------------------------------------------------
# semopy 모델 문법: (잠재변수명) =~ (관측변수1) + (관측변수2) + ...
#                  (종속변수) ~ (독립변수1) + (독립변수2) + ...  => 구조방정식
# 여기서는 일단 HELPfulness를 제외하고 "순수 측정모델"만 정의 (CFA)

model_1_cfa = """
Heuristic =~ Title_Length + Text_Length + Time_Lapsed + Deviation_Of_Star_Ratings + Rating
Systematic =~ Flesch_Reading_Ease + Depth + Breadth + Valence + Arousal + Interaction
"""

# semopy로 모델 생성 & 적합
mod = Model(model_1_cfa)
res = mod.fit(audible)

# (a) 적합도 지표 확인
# semopy의 calc_stats()로 CFI, RMSEA, AIC, etc.를 볼 수 있다
stats = mod.calc_stats()
print(stats)

# (b) 표준화 로딩/계수 등 결과 확인
# 구조방정식 전체 파라미터 요약
estimates = mod.inspect()
print(estimates)

# (c) AVE, CR 계산
# semopy는 내장 함수로 AVE/CR을 계산하는 메서드(semopy.optimize.utils.get_discriminant_validity) 등이 있으나
# 아래처럼 로딩값과 에러분산을 별도 추출해서 수동 계산도 가능.
# 예시로, 잠재변수별로 factor loadings 를 가져와 계산하는 과정:

loadings = estimates.loc[estimates['param'].str.contains('~~') == False]  # "~="는 factor loadings
# 혹은 "Heuristic=~" 식으로 필터링해서 로딩값을 구하고, 에러분산 항 등도 분리한 다음 수식대로 계산



NameError: name 'Model' is not defined

In [ ]:
# -------------------------------------------------
# (2단계) 음이항 회귀(Negative Binomial)
# -------------------------------------------------
# (1단계)에서 얻은 잠재변수 점수(factor scores)를 추출하여
# Helpfulness (count형) 종속변수와 함께 음이항회귀를 시행
factor_scores = mod.predict_factors(audible)  # Heuristic, Systematic 점수가 담긴 DataFrame 반환

df_for_reg = pd.concat([audible['Helpfulness'], factor_scores], axis=1)

# statsmodels에서 음이항(GLM) 지정: family=sm.families.NegativeBinomial()
formula = "Helpfulness ~ Heuristic + Systematic"
nb_model = smf.glm(formula=formula, data=df_for_reg, 
                   family=sm.families.NegativeBinomial()).fit()

print(nb_model.summary())

# (a) AIC 확인
print("AIC:", nb_model.aic)

# (b) McFadden’s R^2 계산
#  - null 모델(상수항만 있는 모델)과 비교
nb_model_null = smf.glm("Helpfulness ~ 1", data=df_for_reg,
                        family=sm.families.NegativeBinomial()).fit()
LL_full = nb_model.llf    # full model log-likelihood
LL_null = nb_model_null.llf   # null model log-likelihood

mcfadden_r2 = 1 - (LL_full / LL_null)
print("McFadden’s R^2:", mcfadden_r2)
